# Project Aether - Google Colab Setup

This notebook sets up and runs Project Aether on Google Colab with GPU support.

## Features
- Automatic GPU detection and setup
- Model downloads (LCM Stable Diffusion)
- All three phases: Probe Training, PPO Training, Evaluation
- Optimized for Colab's T4 GPU (16GB VRAM)


## Step 1: Install Dependencies


In [ ]:
# Install PyTorch with CUDA 12.1 (Colab default)
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121

# Install other dependencies
!pip install diffusers transformers accelerate safetensors
!pip install gymnasium numpy scikit-learn matplotlib tqdm
!pip install pyyaml pillow lpips
!pip install datasets  # For I2P dataset


## Step 2: Clone Repository or Upload Files


In [ ]:
# Option A: Clone from GitHub
!git clone https://github.com/Anastasia-Deniz/project-aether.git
%cd project-aether

# Option B: If you uploaded files manually:
# %cd /content/project-aether


## Step 3: Verify GPU and Setup


In [ ]:
import torch
import sys
from pathlib import Path

# Verify GPU
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Add project to path
sys.path.insert(0, str(Path.cwd()))

# Create necessary directories
!mkdir -p data/latents checkpoints/probes outputs/ppo outputs/evaluation


## Step 4: Phase 1 - Collect Latents


In [ ]:
# Collect latents for probe training
# Colab T4 has 16GB VRAM, so we can use larger batch sizes
# Using --hard_only and higher threshold for better probe accuracy (target: >85%)
!python scripts/collect_latents.py \
    --num_samples 150 \
    --num_steps 8 \
    --device cuda \
    --min_inappropriate_pct 70.0 \
    --hard_only


## Step 5: Phase 1 - Train Probes


In [ ]:
# Train linear probes
# Find the latest latents directory
import os
from pathlib import Path

latents_dirs = sorted(Path('data/latents').glob('run_*'), key=os.path.getmtime)
if latents_dirs:
    latest_latents = latents_dirs[-1]
    print(f"Using latents from: {latest_latents}")
    !python scripts/train_probes.py --latents_dir {latest_latents}
else:
    print("No latents found! Run Step 4 first.")


## Step 6: Phase 2 - Train PPO Policy


In [ ]:
# Train PPO policy with Colab-optimized config
# Colab T4 can handle larger batch sizes than RTX 4050
# The config uses probe_path: "auto" to automatically find the latest probe
!python scripts/train_ppo.py --config configs/colab_optimized.yaml

# Alternative: Manually specify probe path
# import os
# from pathlib import Path
# probe_dirs = sorted(Path('checkpoints/probes').glob('run_*'), key=os.path.getmtime)
# if probe_dirs:
#     latest_probe = probe_dirs[-1] / 'pytorch'
#     !python scripts/train_ppo.py --config configs/colab_optimized.yaml --probe_path {latest_probe}


## Step 7: Phase 3 - Evaluate Policy


In [ ]:
# Evaluate trained policy
import os
from pathlib import Path

# Find latest policy and probe
ppo_dirs = sorted(Path('outputs/ppo').glob('aether_ppo_*'), key=os.path.getmtime)
probe_dirs = sorted(Path('checkpoints/probes').glob('run_*'), key=os.path.getmtime)

if ppo_dirs and probe_dirs:
    latest_policy = ppo_dirs[-1] / 'final_policy.pt'
    latest_probe = probe_dirs[-1] / 'pytorch'
    
    if latest_policy.exists() and latest_probe.exists():
        print(f"Evaluating: {latest_policy}")
        print(f"Using probe: {latest_probe}")
        !python scripts/evaluate_ppo.py \
            --policy_path {latest_policy} \
            --probe_path {latest_probe} \
            --num_samples 30 \
            --device cuda
    else:
        print(f"Policy exists: {latest_policy.exists()}")
        print(f"Probe exists: {latest_probe.exists()}")
else:
    if not ppo_dirs:
        print("No training runs found!")
    if not probe_dirs:
        print("No probe directories found!")


## Save Results to Google Drive

Mount your Google Drive and save results:


In [ ]:
from google.colab import drive
import shutil

# Mount Drive
drive.mount('/content/drive')

# Copy results to Drive
drive_path = '/content/drive/MyDrive/project-aether-results'
!mkdir -p {drive_path}

!cp -r outputs {drive_path}/
!cp -r checkpoints {drive_path}/
!cp -r data/latents {drive_path}/

print(f"Results saved to: {drive_path}")
